In [164]:
import pandas as pd
import numpy as np
pd.set_option('display.max_columns', 100)

In [2]:
# открываем датасет с информацией по дислокации
l2 = pd.read_parquet('train/dislok_wagons.parquet')
l2t = pd.read_parquet('test/dislok_wagons.parquet')
l2 = pd.concat([l2, l2t], ignore_index=True)
del l2t

In [3]:
# формируем колонку 'wagnum'+'month'
l2['wag_m'] = l2['wagnum'].astype('string') + '_' + l2['plan_date'].dt.month.astype('string')

In [4]:
# новые колонки:
# dep: сколько дней до деповского ремонта в днях
# kap: сколько дней до капитального ремонта в днях
# plan: сколько дней до планового ремонта в днях
l2['dep'] = (l2['plan_date'] - l2['date_pl_rem']).dt.days
l2['kap'] = (l2['plan_date'] - l2['date_kap']).dt.days
l2['plan'] = (l2['plan_date'] - l2['date_pl_rem']).dt.days
l2.head(3)

,plan_date,wagnum,date_kap,date_dep,kod_vrab,date_pl_rem,id_road_disl,st_id_dest,id_road_dest,st_id_send,id_road_send,ost_prob,isload,fr_id,last_fr_id,distance,wag_m,dep,kap,plan
0,2022-08-01,1071,NaT,2020-02-16,0,2023-02-16,42,22,42,11470,42,36899,0,5032,1413,-1.0,1071_8,-199,NaN,-199
1,2022-08-02,1071,NaT,2020-02-16,0,2023-02-16,42,11663,0,11470,42,36899,1,1204,1204,998.0,1071_8,-198,NaN,-198
2,2022-08-03,1071,NaT,2020-02-16,0,2023-02-16,42,11663,0,11470,42,36711,1,1204,1204,989.0,1071_8,-197,NaN,-197


In [5]:
# формируем новый целевой датафрейм. агрегируем
df = l2.groupby('wag_m')[['dep','kap','plan','isload']].mean()

In [6]:
# добавляем самый частый код груза и код последнего груза для каждого месяца использования вагона
def ser_mode(s):
    return s.mode()[0]
df[['fr_id','last_fr_id']] = l2.groupby(['wag_m'])[['fr_id','last_fr_id']].agg([ser_mode])
df = df.astype({'fr_id':'category','last_fr_id':'category'})
df.head(3)

,dep,kap,plan,isload,fr_id,last_fr_id
wag_m,,,,,,
0_1,-110.0,3451.0,-110.0,0.16129,2255,3381
0_10,-202.0,3359.0,-202.0,0.096774,5033,1475
0_11,-171.5,3389.5,-171.5,0.5,1061,1061


In [7]:
l2 = np.nan
del l2

In [8]:
# открываем список вагонов, по которым известен пробег и тип владения на дату среза
l1 = pd.read_parquet('train/wagons_probeg_ownersip.parquet')
l1t = pd.read_parquet('test/wagons_probeg_ownersip.parquet')
l1 = pd.concat([l1, l1t], ignore_index=True)
del l1t

In [9]:
# формируем колонку 'wagnum'+'month'
l1['wag_m'] = l1['wagnum'].astype('string') + '_' + l1['month'].astype('string')

In [10]:
# создаем колонку ost - на сколько за месяц уменьшился остаточный пробег
l1_gr = l1.groupby('wag_m')[['ost_prob']].max()
l1_gr['ost_min'] = l1.groupby('wag_m')[['ost_prob']].min()
l1_gr['ost'] = l1_gr['ost_prob'] - l1_gr['ost_min']

In [11]:
# джойним колонку ost к нашему целевому фрейму
df_2 = df.merge(l1_gr[['ost']], how='left', on='wag_m')
df_2.head(3)

,dep,kap,plan,isload,fr_id,last_fr_id,ost
wag_m,,,,,,,
0_1,-110.0,3451.0,-110.0,0.16129,2255,3381,4158
0_10,-202.0,3359.0,-202.0,0.096774,5033,1475,995
0_11,-171.5,3389.5,-171.5,0.5,1061,1061,5691


In [12]:
l1,l1_gr,df = np.nan,np.nan,np.nan
del l1,l1_gr,df

In [13]:
# открываем данные по текущим ремонтам вагона
l5 = pd.read_parquet('train/tr_rems.parquet')
l5t = pd.read_parquet('test/tr_rems.parquet')
l5 = pd.concat([l5, l5t], ignore_index=True)
del l5t

In [14]:
# создаем колонку wag_m - месяц ремонта вагона
# col - флаг ремонта
l5['wag_m'] = l5['wagnum'].astype('string') + '_' + l5['rem_month'].dt.month.astype('string')
l5['col'] = 1
# l5.drop_duplicates(inplace=True) # предположим, что дублей нет. это несколько ремонтов в месяц

In [15]:
# агрегируем по wag_m. узнаем по col: сколько ремонтов было за месяц
l5_gr = l5.groupby('wag_m')[['col']].count()

In [16]:
# джойним колонку col к целевому фрейму. преобразуем к col_res
df_3 = df_2.copy()
df_3['col_res'] = 0
df_3 = df_3.merge(l5_gr[['col']], how='left', on='wag_m')
df_3.loc[~df_3['col'].isna(), 'col_res'] = df_3['col']
df_3.drop('col', inplace=True, axis=1)
df_3 = df_3.astype({'col_res': 'category'})

In [17]:
# агрегируем по wag_m. определяем kod_vrab, gr_probeg, por_probeg
l5_gr2 = l5.groupby('wag_m')[['kod_vrab','gr_probeg','por_probeg']].max()

In [18]:
# джойним колонки kod_vrab, gr_probeg, por_probeg к целевому фрейму
df_3['vrab'] = 0
df_3 = df_3.merge(l5_gr2[['kod_vrab','gr_probeg','por_probeg']], how='left', on='wag_m')
df_3.loc[~df_3['kod_vrab'].isna(), 'vrab'] = df_3['kod_vrab']
df_3.drop('kod_vrab', inplace=True, axis=1)
df_3 = df_3.astype({'vrab': 'category'})
df_3.head()

,dep,kap,plan,isload,fr_id,last_fr_id,ost,col_res,vrab,gr_probeg,por_probeg
wag_m,,,,,,,,,,,
0_1,-110.0,3451.0,-110.0,0.16129,2255,3381,4158,0,0,<NA>,<NA>
0_10,-202.0,3359.0,-202.0,0.096774,5033,1475,995,0,0,<NA>,<NA>
0_11,-171.5,3389.5,-171.5,0.5,1061,1061,5691,1,3,72499,24311
0_12,-141.0,3420.0,-141.0,0.419355,5032,1061,5448,1,3,72499,24448
0_2,-80.5,3480.5,-80.5,0.607143,2255,536,10230,0,0,<NA>,<NA>


In [19]:
l5_gr2,l5_gr,l5,df_2 = np.nan,np.nan,np.nan,np.nan
del l5_gr2,l5_gr,l5,df_2

In [20]:
# преобразуем gr_probeg и por_probeg в категориальные переменные
# наверное есть что-то красивое встроенное в pandas, но нет времени искать:)
def num_to_cat(frame, col):
    x2 = frame.copy()
    q1 = x2[col].quantile(q=0.33)
    q2 = x2[col].quantile(q=0.67)
    x2['cat_'+col] = 0
    x2.loc[((~x2[col].isna()) & (x2[col] < q1)), 'cat_'+col] = 1
    x2.loc[((~x2[col].isna()) & (x2[col] > q1) & (x2[col] < q2)), 'cat_'+col] = 2
    x2.loc[((~x2[col].isna()) & (x2[col] > q2)), 'cat_'+col] = 3
    x2 = x2.astype({'cat_'+col: 'category'})
    x2.drop(col, inplace=True, axis=1)
    return x2


df_4 = df_3.copy()
df_4 = num_to_cat(df_4, 'gr_probeg')
df_4 = num_to_cat(df_4, 'por_probeg')

In [21]:
df_3 = np.nan
del df_3

In [22]:
# открываем данные по параметрам вагона
l3 = pd.read_parquet('train/wag_params.parquet')

In [23]:
# приписываем к целевому фрейму колонку wagnum. потом джойним с l3, чтобы приписать модель
df_4['wagnum'] = df_4.index.str.split('_').str[0].astype('int')
df_5 = df_4.reset_index().merge(l3[['model','wagnum']], how='left', on='wagnum')
df_5 = df_5.astype({'model': 'category'})
df_5.drop('wagnum', inplace=True, axis=1)
df_5.head(3)

,wag_m,dep,kap,plan,isload,fr_id,last_fr_id,ost,col_res,vrab,cat_gr_probeg,cat_por_probeg,model
0,0_1,-110.0,3451.0,-110.0,0.16129,2255,3381,4158,0,0,0,0,11-276
1,0_10,-202.0,3359.0,-202.0,0.096774,5033,1475,995,0,0,0,0,11-276
2,0_11,-171.5,3389.5,-171.5,0.5,1061,1061,5691,1,3,2,2,11-276


In [24]:
df_4,l3 = np.nan,np.nan
del df_4,l3

In [25]:
# создаем новые фичи: переносим в каждую строку данные за предыдущие 3 месяца
def shift(c):
    c1 = int(c)
    if c1 == 1:
        return '12'
    else:
        return str(c1-1)

def shift_foo(frame,col):
    x2 = frame.copy()
    for i in range(3):
        x_t1 = x2[['wag_m',col]].copy()
        x_t1.rename(columns={'wag_m':'w_'+str(i+1),col:col+'-'+str(i+1)},inplace=True)
        d_n = x2.merge(x_t1,how='left',on='w_'+str(i+1))
        x2 = d_n.copy()
    return x2

columns_transform = [
    'dep',
    'kap',
    'plan',
    'isload',
    'fr_id',
    'last_fr_id',
    'ost',
    'col_res',
    'vrab',
    'cat_gr_probeg',
    'cat_por_probeg']
        
df_6 = df_5.copy()
df_6['w_1'] = df_6['wag_m'].str.split('_').str[0] + '_' + df_6['wag_m'].str.split('_').str[1].apply(shift)
df_6['w_2'] = df_6['w_1'].str.split('_').str[0] + '_' + df_6['w_1'].str.split('_').str[1].apply(shift)
df_6['w_3'] = df_6['w_2'].str.split('_').str[0] + '_' + df_6['w_2'].str.split('_').str[1].apply(shift)

for i in columns_transform:
    df_6 = shift_foo(df_6, i)

df_6.head(3)

,wag_m,dep,kap,plan,isload,fr_id,last_fr_id,ost,col_res,vrab,cat_gr_probeg,cat_por_probeg,model,w_1,w_2,w_3,dep-1,dep-2,dep-3,kap-1,kap-2,kap-3,plan-1,plan-2,plan-3,isload-1,isload-2,isload-3,fr_id-1,fr_id-2,fr_id-3,last_fr_id-1,last_fr_id-2,last_fr_id-3,ost-1,ost-2,ost-3,col_res-1,col_res-2,col_res-3,vrab-1,vrab-2,vrab-3,cat_gr_probeg-1,cat_gr_probeg-2,cat_gr_probeg-3,cat_por_probeg-1,cat_por_probeg-2,cat_por_probeg-3
0,0_1,-110.0,3451.0,-110.0,0.16129,2255,3381,4158,0,0,0,0,11-276,0_12,0_11,0_10,-141.0,-171.5,-202.0,3420.0,3389.5,3359.0,-141.0,-171.5,-202.0,0.419355,0.5,0.096774,5032,1061,5033,1061,1061,1475,5448,5691,995,1,1,0,3,3,0,2,2,0,2,2,0
1,0_10,-202.0,3359.0,-202.0,0.096774,5033,1475,995,0,0,0,0,11-276,0_9,0_8,0_7,-232.5,-263.0,NaN,3328.5,3298.0,NaN,-232.5,-263.0,NaN,0.6,0.322581,<NA>,2255,2255,NaN,1671,1671,NaN,5446,7091,<NA>,0,0,NaN,0,0,NaN,0,0,NaN,0,0,NaN
2,0_11,-171.5,3389.5,-171.5,0.5,1061,1061,5691,1,3,2,2,11-276,0_10,0_9,0_8,-202.0,-232.5,-263.0,3359.0,3328.5,3298.0,-202.0,-232.5,-263.0,0.096774,0.6,0.322581,5033,2255,2255,1475,1671,1671,995,5446,7091,0,0,0,0,0,0,0,0,0,0,0,0


In [26]:
# чистим датасет, удаляем март, сентябрь и август
# удаляем колонки, связанные с текущим месяцем
df_7 = df_6.copy()
df_7['month'] = df_7['wag_m'].str.split('_').str[1].astype('int')
df_7.drop(df_7.loc[df_7['month'] == 3].index, axis = 0, inplace = True)
df_7.drop(df_7.loc[df_7['month'] == 8].index, axis = 0, inplace = True)
df_7.drop(df_7.loc[df_7['month'] == 9].index, axis = 0, inplace = True)

df_7.drop(columns_transform, axis = 1, inplace = True)
df_7.drop('month', axis = 1, inplace = True)
df_7.head(3)

,wag_m,model,w_1,w_2,w_3,dep-1,dep-2,dep-3,kap-1,kap-2,kap-3,plan-1,plan-2,plan-3,isload-1,isload-2,isload-3,fr_id-1,fr_id-2,fr_id-3,last_fr_id-1,last_fr_id-2,last_fr_id-3,ost-1,ost-2,ost-3,col_res-1,col_res-2,col_res-3,vrab-1,vrab-2,vrab-3,cat_gr_probeg-1,cat_gr_probeg-2,cat_gr_probeg-3,cat_por_probeg-1,cat_por_probeg-2,cat_por_probeg-3
0,0_1,11-276,0_12,0_11,0_10,-141.0,-171.5,-202.0,3420.0,3389.5,3359.0,-141.0,-171.5,-202.0,0.419355,0.5,0.096774,5032,1061,5033,1061,1061,1475,5448,5691,995,1,1,0,3,3,0,2,2,0,2,2,0
1,0_10,11-276,0_9,0_8,0_7,-232.5,-263.0,NaN,3328.5,3298.0,NaN,-232.5,-263.0,NaN,0.6,0.322581,<NA>,2255,2255,NaN,1671,1671,NaN,5446,7091,<NA>,0,0,NaN,0,0,NaN,0,0,NaN,0,0,NaN
2,0_11,11-276,0_10,0_9,0_8,-202.0,-232.5,-263.0,3359.0,3328.5,3298.0,-202.0,-232.5,-263.0,0.096774,0.6,0.322581,5033,2255,2255,1475,1671,1671,995,5446,7091,0,0,0,0,0,0,0,0,0,0,0,0


In [27]:
df_6 = np.nan
del df_6

In [28]:
# добавляем колонки с проведенным плановым ремонтом
y_test = pd.read_csv('test/target/y_test.csv')
y_train = pd.read_csv('train/target/y_train.csv')
df_8 = df_7.copy()

In [29]:
def train_test(frame):
    x2 = frame.copy()
    x2['month']= pd.to_datetime(x2['month'])
    x2['month_n'] = x2['month'].dt.month.astype('string')
    x2['wag_m'] = x2['wagnum'].astype('string') + '_' + x2['month_n']
    return x2

y_test1 = train_test(y_test)
y_train1 = train_test(y_train)

In [30]:
df_9 = df_8.merge(y_test1[['target_month','target_day','wag_m']], how='left', on='wag_m')
df_9['target'] = df_9['target_month'] + df_9['target_day']
df_9.drop(['target_month','target_day'], axis=1, inplace=True)
df_10 = df_9.merge(y_train1[['target_month','target_day','wag_m']], how='left', on='wag_m')
df_10.loc[df_10['target'].isna(), 'target'] = df_10['target_month'] + df_10['target_day']
df_10.drop(['target_month','target_day'], axis=1, inplace=True)
df_10 = df_10.astype({'target': 'category'})
df_10.head(3)

,wag_m,model,w_1,w_2,w_3,dep-1,dep-2,dep-3,kap-1,kap-2,kap-3,plan-1,plan-2,plan-3,isload-1,isload-2,isload-3,fr_id-1,fr_id-2,fr_id-3,last_fr_id-1,last_fr_id-2,last_fr_id-3,ost-1,ost-2,ost-3,col_res-1,col_res-2,col_res-3,vrab-1,vrab-2,vrab-3,cat_gr_probeg-1,cat_gr_probeg-2,cat_gr_probeg-3,cat_por_probeg-1,cat_por_probeg-2,cat_por_probeg-3,target
0,0_1,11-276,0_12,0_11,0_10,-141.0,-171.5,-202.0,3420.0,3389.5,3359.0,-141.0,-171.5,-202.0,0.419355,0.5,0.096774,5032,1061,5033,1061,1061,1475,5448,5691,995,1,1,0,3,3,0,2,2,0,2,2,0,0.0
1,0_10,11-276,0_9,0_8,0_7,-232.5,-263.0,NaN,3328.5,3298.0,NaN,-232.5,-263.0,NaN,0.6,0.322581,<NA>,2255,2255,NaN,1671,1671,NaN,5446,7091,<NA>,0,0,NaN,0,0,NaN,0,0,NaN,0,0,NaN,0.0
2,0_11,11-276,0_10,0_9,0_8,-202.0,-232.5,-263.0,3359.0,3328.5,3298.0,-202.0,-232.5,-263.0,0.096774,0.6,0.322581,5033,2255,2255,1475,1671,1671,995,5446,7091,0,0,0,0,0,0,0,0,0,0,0,0,0.0


In [31]:
y_test1,y_train1,y_test,y_train = np.nan,np.nan,np.nan,np.nan
del y_test1,y_train1,y_test,y_train

In [32]:
# готовим фрейм к ml: удаляем колонки w_1, w_2, w_3 и строки с target=NaN
df_11 = df_10.copy()
df_11.drop(['w_1','w_2','w_3'], inplace=True, axis=1)
df_11.dropna(axis=0, how='any', subset=['target'], inplace=True)
df_11.head(3)

,wag_m,model,dep-1,dep-2,dep-3,kap-1,kap-2,kap-3,plan-1,plan-2,plan-3,isload-1,isload-2,isload-3,fr_id-1,fr_id-2,fr_id-3,last_fr_id-1,last_fr_id-2,last_fr_id-3,ost-1,ost-2,ost-3,col_res-1,col_res-2,col_res-3,vrab-1,vrab-2,vrab-3,cat_gr_probeg-1,cat_gr_probeg-2,cat_gr_probeg-3,cat_por_probeg-1,cat_por_probeg-2,cat_por_probeg-3,target
0,0_1,11-276,-141.0,-171.5,-202.0,3420.0,3389.5,3359.0,-141.0,-171.5,-202.0,0.419355,0.5,0.096774,5032,1061,5033,1061,1061,1475,5448,5691,995,1,1,0,3,3,0,2,2,0,2,2,0,0.0
1,0_10,11-276,-232.5,-263.0,NaN,3328.5,3298.0,NaN,-232.5,-263.0,NaN,0.6,0.322581,<NA>,2255,2255,NaN,1671,1671,NaN,5446,7091,<NA>,0,0,NaN,0,0,NaN,0,0,NaN,0,0,NaN,0.0
2,0_11,11-276,-202.0,-232.5,-263.0,3359.0,3328.5,3298.0,-202.0,-232.5,-263.0,0.096774,0.6,0.322581,5033,2255,2255,1475,1671,1671,995,5446,7091,0,0,0,0,0,0,0,0,0,0,0,0,0.0


In [33]:
# уравновешиваем количество классификаций target
df_13 = df_11.loc[df_11['target'] == 2]
df_13 = df_11.loc[df_13.index.repeat(59)]
df_13 = pd.concat([df_11.loc[df_11['target']==0], df_13]).sample(frac=1)
df_14 = df_11.loc[df_11['target'] == 1]
df_14 = df_14.loc[df_14.index.repeat(28)]
df_14 = pd.concat([df_13, df_14]).sample(frac=1)
df_14.head(3)

,wag_m,model,dep-1,dep-2,dep-3,kap-1,kap-2,kap-3,plan-1,plan-2,plan-3,isload-1,isload-2,isload-3,fr_id-1,fr_id-2,fr_id-3,last_fr_id-1,last_fr_id-2,last_fr_id-3,ost-1,ost-2,ost-3,col_res-1,col_res-2,col_res-3,vrab-1,vrab-2,vrab-3,cat_gr_probeg-1,cat_gr_probeg-2,cat_gr_probeg-3,cat_por_probeg-1,cat_por_probeg-2,cat_por_probeg-3,target
85142,25333_2,12-132-03,-39.0,-70.0,-100.5,1057.0,1026.0,995.5,-39.0,-70.0,-100.5,0.258065,0.419355,0.366667,2255,727,5032,1639,727,1593,1833,5072,162294,0,0,0,0,0,0,0,0,0,0,0,0,0.0
82289,24820_10,12-132,-341.5,-372.0,NaN,NaN,NaN,NaN,-341.5,-372.0,NaN,0.233333,0.483871,<NA>,1655,2255,NaN,1655,1045,NaN,1095,2871,<NA>,0,0,NaN,0,0,NaN,0,0,NaN,0,0,NaN,1.0
121710,31917_10,12-1303-01,-377.5,-408.0,NaN,717.5,687.0,NaN,-377.5,-408.0,NaN,0.5,0.16129,<NA>,615,2255,NaN,615,714,NaN,4457,4425,<NA>,0,0,NaN,0,0,NaN,0,0,NaN,0,0,NaN,2.0


In [34]:
# df_14.to_csv('my_df.csv',index=False)

In [34]:
df_15 = df_14.drop('wag_m', axis=1)

In [35]:
df_15.info()

<class 'pandas.core.frame.DataFrame'>
Index: 481540 entries, 85142 to 141827
Data columns (total 35 columns):
 #   Column            Non-Null Count   Dtype   
---  ------            --------------   -----   
 0   model             481540 non-null  category
 1   dep-1             481540 non-null  float64 
 2   dep-2             481511 non-null  float64 
 3   dep-3             381633 non-null  float64 
 4   kap-1             251897 non-null  float64 
 5   kap-2             252392 non-null  float64 
 6   kap-3             199910 non-null  float64 
 7   plan-1            481540 non-null  float64 
 8   plan-2            481511 non-null  float64 
 9   plan-3            381633 non-null  float64 
 10  isload-1          481540 non-null  Float64 
 11  isload-2          481511 non-null  Float64 
 12  isload-3          381633 non-null  Float64 
 13  fr_id-1           481540 non-null  category
 14  fr_id-2           481511 non-null  category
 15  fr_id-3           381633 non-null  category
 16  las

In [36]:
del df_7,df_8,df_9,df_10,df_11,df_13

In [37]:
# обучаем модель
import dill
from sklearn.pipeline import make_pipeline
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import FunctionTransformer, OneHotEncoder, QuantileTransformer
from sklearn.compose import ColumnTransformer, make_column_selector
from sklearn.impute import SimpleImputer
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split, GridSearchCV

import numpy as np

In [ ]:
# удалось обучить только LogisticRegression со стандартными настройками. на остальное не хватает памяти

In [73]:
df_16 = df_15.copy()
x, y = df_16.drop(['target'], axis=1), df_15['target']
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.3, random_state=0)

column_transformer = ColumnTransformer([
    ('numerical_transformer',
     make_pipeline(
         SimpleImputer(strategy='mean'),
         QuantileTransformer(),
     ),
     make_column_selector(dtype_include=np.number)
     ),
    ('categorical_transformer',
     make_pipeline(
         SimpleImputer(strategy='most_frequent'),
         OneHotEncoder(sparse_output=False, handle_unknown='ignore'),
     ),
     make_column_selector(dtype_include=['object','category']) 
     ),
], remainder='passthrough')

preprocessor = Pipeline(steps=[    
    ('column_transformer', column_transformer),
    ('model', SVC())
])

params = {'model': [
    # LogisticRegression(max_iter=100000, solver='lbfgs'),
    RandomForestClassifier(),
    # SVC(),
]}

grid = GridSearchCV(preprocessor, params, cv=4, n_jobs=-1)
grid.fit(x_train, y_train)
print(grid.best_params_, grid.score(x_test, y_test))

with open('pipe_forest.pkl', 'wb') as dill_file:
    dill.dump(grid, dill_file)


KeyboardInterrupt: 

In [38]:
with open('pipe_logreg.pkl' , 'rb') as f:
    model = dill.load(f)

In [ ]:
# формируем prediction
# а дальше не хватило времени(

In [41]:
y_predict_submit_example = pd.read_csv('train/target/y_predict_submit_example.csv')
y_predict_submit_example

,wagnum,month,target_month,target_day
0,33361,2023-03-01,1,1
1,33364,2023-03-01,1,1
2,33366,2023-03-01,1,1
3,33358,2023-03-01,1,1
4,33349,2023-03-01,1,1
...,...,...,...,...
33702,17621,2023-03-01,1,1
33703,25045,2023-03-01,1,1
33704,27156,2023-03-01,1,1
33705,21361,2023-03-01,1,1


In [42]:
df_14

,wag_m,model,dep-1,dep-2,dep-3,kap-1,kap-2,kap-3,plan-1,plan-2,plan-3,isload-1,isload-2,isload-3,fr_id-1,fr_id-2,fr_id-3,last_fr_id-1,last_fr_id-2,last_fr_id-3,ost-1,ost-2,ost-3,col_res-1,col_res-2,col_res-3,vrab-1,vrab-2,vrab-3,cat_gr_probeg-1,cat_gr_probeg-2,cat_gr_probeg-3,cat_por_probeg-1,cat_por_probeg-2,cat_por_probeg-3,target
85142,25333_2,12-132-03,-39.0,-70.0,-100.5,1057.0,1026.0,995.5,-39.0,-70.0,-100.5,0.258065,0.419355,0.366667,2255,727,5032,1639,727,1593,1833,5072,162294,0,0,0,0,0,0,0,0,0,0,0,0,0.0
82289,24820_10,12-132,-341.5,-372.0,NaN,NaN,NaN,NaN,-341.5,-372.0,NaN,0.233333,0.483871,<NA>,1655,2255,NaN,1655,1045,NaN,1095,2871,<NA>,0,0,NaN,0,0,NaN,0,0,NaN,0,0,NaN,1.0
121710,31917_10,12-1303-01,-377.5,-408.0,NaN,717.5,687.0,NaN,-377.5,-408.0,NaN,0.5,0.16129,<NA>,615,2255,NaN,615,714,NaN,4457,4425,<NA>,0,0,NaN,0,0,NaN,0,0,NaN,0,0,NaN,2.0
149226,6300_12,11-276,-510.5,-541.0,-571.5,3767.5,3737.0,3706.5,-510.5,-541.0,-571.5,0.466667,0.419355,0.0,1475,1413,2255,1413,1413,4987,1359,2934,138,0,0,0,0,0,0,0,0,0,0,0,0,0.0
64750,21663_11,12-132,-677.0,-707.5,-738.0,419.0,388.5,358.0,-677.0,-707.5,-738.0,0.548387,0.533333,0.483871,1606,617,615,1606,617,1655,5078,7958,2703,0,0,0,0,0,0,0,0,0,0,0,0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
60598,20916_1,12-9788-01,-390.0,-420.5,-451.0,705.0,674.5,644.0,-390.0,-420.5,-451.0,0.483871,0.6,0.322581,2255,714,2255,714,714,617,12344,2077,5916,0,0,0,0,0,0,0,0,0,0,0,0,0.0
32336,15822_2,11-280,-622.0,-653.0,-683.5,474.0,443.0,412.5,-622.0,-653.0,-683.5,0.0,0.612903,0.5,2255,3689,2255,3689,3689,365,302,3045,1605,0,0,0,0,0,0,0,0,0,0,0,0,0.0
89488,26116_1,12-132-03,-340.0,-370.5,-401.0,NaN,NaN,NaN,-340.0,-370.5,-401.0,0.258065,0.833333,0.354839,2255,629,2255,1593,629,748,5847,7780,405,0,0,0,0,0,0,0,0,0,0,0,0,2.0
35726,16434_2,11-1807-01,184.0,153.0,122.5,NaN,NaN,NaN,184.0,153.0,122.5,0.064516,0.774194,0.533333,2255,3381,2334,1023,3381,2334,886,4282,2850,0,0,0,0,0,0,0,0,0,0,0,0,0.0


In [194]:
pr_1 = y_predict_submit_example.copy()

In [195]:
pr_1['month_d'] = pd.to_datetime(pr_1['month'])

In [196]:
pr_1

,wagnum,month,target_month,target_day,month_d
0,33361,2023-03-01,1,1,2023-03-01
1,33364,2023-03-01,1,1,2023-03-01
2,33366,2023-03-01,1,1,2023-03-01
3,33358,2023-03-01,1,1,2023-03-01
4,33349,2023-03-01,1,1,2023-03-01
...,...,...,...,...,...
33702,17621,2023-03-01,1,1,2023-03-01
33703,25045,2023-03-01,1,1,2023-03-01
33704,27156,2023-03-01,1,1,2023-03-01
33705,21361,2023-03-01,1,1,2023-03-01


In [214]:
pr_1['month_n'] = pr_1['month_d'].dt.month.astype('string')
pr_1['wag_m'] = pr_1['wagnum'].astype('string') + '_2'

In [215]:
pr_1

,wagnum,month,target_month,target_day,month_d,month_n,wag_m
0,33361,2023-03-01,1,1,2023-03-01,3,33361_2
1,33364,2023-03-01,1,1,2023-03-01,3,33364_2
2,33366,2023-03-01,1,1,2023-03-01,3,33366_2
3,33358,2023-03-01,1,1,2023-03-01,3,33358_2
4,33349,2023-03-01,1,1,2023-03-01,3,33349_2
...,...,...,...,...,...,...,...
33702,17621,2023-03-01,1,1,2023-03-01,3,17621_2
33703,25045,2023-03-01,1,1,2023-03-01,3,25045_2
33704,27156,2023-03-01,1,1,2023-03-01,3,27156_2
33705,21361,2023-03-01,1,1,2023-03-01,3,21361_2


In [199]:
pr_2 = pr_1.merge(df_14, how='left', on='wag_m')

In [200]:
pr_2

,wagnum,month,target_month,target_day,month_d,month_n,wag_m,model,dep-1,dep-2,dep-3,kap-1,kap-2,kap-3,plan-1,plan-2,plan-3,isload-1,isload-2,isload-3,fr_id-1,fr_id-2,fr_id-3,last_fr_id-1,last_fr_id-2,last_fr_id-3,ost-1,ost-2,ost-3,col_res-1,col_res-2,col_res-3,vrab-1,vrab-2,vrab-3,cat_gr_probeg-1,cat_gr_probeg-2,cat_gr_probeg-3,cat_por_probeg-1,cat_por_probeg-2,cat_por_probeg-3,target
0,33361,2023-03-01,1,1,2023-03-01,3,33361_2,12-1303-01,-32.0,-63.0,-93.5,1064.0,1033.0,1002.5,-32.0,-63.0,-93.5,1.0,0.483871,0.066667,711,1593,5032,711,1593,609,2152,6224,159366,0,0,0,0,0,0,0,0,0,0,0,0,0.0
1,33364,2023-03-01,1,1,2023-03-01,3,33364_2,12-1303-01,-260.0,-291.0,-321.5,835.0,804.0,773.5,-260.0,-291.0,-321.5,0.290323,0.193548,0.4,714,2255,2255,714,714,714,8994,158204,12453,0,2,0,0,5,0,0,3,0,0,3,0,0.0
2,33366,2023-03-01,1,1,2023-03-01,3,33366_2,12-1303-01,-77.0,-108.0,-138.5,NaN,NaN,NaN,-77.0,-108.0,-138.5,0.451613,0.0,0.233333,1660,5032,1083,1660,619,1083,4757,158703,2013,0,0,0,0,0,0,0,0,0,0,0,0,0.0
3,33358,2023-03-01,1,1,2023-03-01,3,33358_2,12-1303-01,-403.0,-434.0,-464.5,692.0,661.0,630.5,-403.0,-434.0,-464.5,0.354839,0.387097,0.066667,2255,1639,2255,1639,1639,1636,3738,1626,1366,0,0,0,0,0,0,0,0,0,0,0,0,0.0
4,33349,2023-03-01,1,1,2023-03-01,3,33349_2,12-1303-01,-171.0,-202.0,-232.5,924.0,893.0,862.5,-171.0,-202.0,-232.5,0.645161,0.451613,0.5,1639,1639,615,1639,1639,615,5835,3796,3972,0,0,0,0,0,0,0,0,0,0,0,0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
93841,17621,2023-03-01,1,1,2023-03-01,3,17621_2,12-132,-663.0,-694.0,-724.5,433.0,402.0,371.5,-663.0,-694.0,-724.5,0.419355,0.645161,0.366667,2255,722,2255,722,722,714,6687,7026,6057,0,0,0,0,0,0,0,0,0,0,0,0,0.0
93842,25045,2023-03-01,1,1,2023-03-01,3,25045_2,12-132,-164.0,-195.0,-225.5,931.0,900.0,869.5,-164.0,-195.0,-225.5,0.322581,0.677419,0.533333,1160,1160,1655,1160,1655,1655,3387,6207,5619,0,0,0,0,0,0,0,0,0,0,0,0,0.0
93843,27156,2023-03-01,1,1,2023-03-01,3,27156_2,12-132,-162.0,-193.0,-223.5,933.0,902.0,871.5,-162.0,-193.0,-223.5,0.387097,0.483871,0.6,2255,714,640,722,714,640,13152,6880,4744,0,0,0,0,0,0,0,0,0,0,0,0,0.0
93844,21361,2023-03-01,1,1,2023-03-01,3,21361_2,12-783,-143.0,-174.0,-204.5,2330.0,2299.0,2268.5,-143.0,-174.0,-204.5,0.516129,0.483871,0.5,722,640,1593,722,640,1593,6416,5022,4571,0,0,0,0,0,0,0,0,0,0,0,0,0.0


In [201]:
pr_2 = pr_2.iloc[:33707,:].copy()

In [202]:
pr_3 = pr_2.drop(['month','target_month','target_day','month_n','wag_m','target','month_d','wagnum'], axis=1)

In [203]:
pr_3
pr_4 = pr_3.bfill()

In [204]:
m = model.predict(pr_4)

In [205]:
m1 = pd.DataFrame(data=m, columns=['A'])

In [117]:
m1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 93846 entries, 0 to 93845
Data columns (total 1 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   A       93846 non-null  float64
dtypes: float64(1)
memory usage: 733.3 KB


In [206]:
# pr_5 = pd.concat([pr_4,pr_2['wagnum']],axis=1)
pr_5 = pd.concat([pr_2,m1],axis=1)

In [207]:
pr_5

,wagnum,month,target_month,target_day,month_d,month_n,wag_m,model,dep-1,dep-2,dep-3,kap-1,kap-2,kap-3,plan-1,plan-2,plan-3,isload-1,isload-2,isload-3,fr_id-1,fr_id-2,fr_id-3,last_fr_id-1,last_fr_id-2,last_fr_id-3,ost-1,ost-2,ost-3,col_res-1,col_res-2,col_res-3,vrab-1,vrab-2,vrab-3,cat_gr_probeg-1,cat_gr_probeg-2,cat_gr_probeg-3,cat_por_probeg-1,cat_por_probeg-2,cat_por_probeg-3,target,A
0,33361,2023-03-01,1,1,2023-03-01,3,33361_2,12-1303-01,-32.0,-63.0,-93.5,1064.0,1033.0,1002.5,-32.0,-63.0,-93.5,1.0,0.483871,0.066667,711,1593,5032,711,1593,609,2152,6224,159366,0,0,0,0,0,0,0,0,0,0,0,0,0.0,2.0
1,33364,2023-03-01,1,1,2023-03-01,3,33364_2,12-1303-01,-260.0,-291.0,-321.5,835.0,804.0,773.5,-260.0,-291.0,-321.5,0.290323,0.193548,0.4,714,2255,2255,714,714,714,8994,158204,12453,0,2,0,0,5,0,0,3,0,0,3,0,0.0,2.0
2,33366,2023-03-01,1,1,2023-03-01,3,33366_2,12-1303-01,-77.0,-108.0,-138.5,NaN,NaN,NaN,-77.0,-108.0,-138.5,0.451613,0.0,0.233333,1660,5032,1083,1660,619,1083,4757,158703,2013,0,0,0,0,0,0,0,0,0,0,0,0,0.0,2.0
3,33358,2023-03-01,1,1,2023-03-01,3,33358_2,12-1303-01,-403.0,-434.0,-464.5,692.0,661.0,630.5,-403.0,-434.0,-464.5,0.354839,0.387097,0.066667,2255,1639,2255,1639,1639,1636,3738,1626,1366,0,0,0,0,0,0,0,0,0,0,0,0,0.0,2.0
4,33349,2023-03-01,1,1,2023-03-01,3,33349_2,12-1303-01,-171.0,-202.0,-232.5,924.0,893.0,862.5,-171.0,-202.0,-232.5,0.645161,0.451613,0.5,1639,1639,615,1639,1639,615,5835,3796,3972,0,0,0,0,0,0,0,0,0,0,0,0,0.0,2.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
33702,2088,2023-03-01,1,1,2023-03-01,3,2088_2,11-2135-01,-49.0,-80.0,-110.5,NaN,NaN,NaN,-49.0,-80.0,-110.5,0.290323,0.483871,0.5,2255,2255,536,1671,2921,536,<NA>,<NA>,<NA>,0,0,0,0,0,0,0,0,0,0,0,0,1.0,2.0
33703,2088,2023-03-01,1,1,2023-03-01,3,2088_2,11-2135-01,-49.0,-80.0,-110.5,NaN,NaN,NaN,-49.0,-80.0,-110.5,0.290323,0.483871,0.5,2255,2255,536,1671,2921,536,<NA>,<NA>,<NA>,0,0,0,0,0,0,0,0,0,0,0,0,1.0,2.0
33704,2088,2023-03-01,1,1,2023-03-01,3,2088_2,11-2135-01,-49.0,-80.0,-110.5,NaN,NaN,NaN,-49.0,-80.0,-110.5,0.290323,0.483871,0.5,2255,2255,536,1671,2921,536,<NA>,<NA>,<NA>,0,0,0,0,0,0,0,0,0,0,0,0,1.0,2.0
33705,2088,2023-03-01,1,1,2023-03-01,3,2088_2,11-2135-01,-49.0,-80.0,-110.5,NaN,NaN,NaN,-49.0,-80.0,-110.5,0.290323,0.483871,0.5,2255,2255,536,1671,2921,536,<NA>,<NA>,<NA>,0,0,0,0,0,0,0,0,0,0,0,0,1.0,2.0


In [208]:
pr_6 = pr_5[['wagnum','month','A']]

In [209]:
pr_6['target_month'] = 0
pr_6['target_day'] = 0

/var/folders/tf/3bx8pk1n20s1ls73n0mjlrf40000gn/T/ipykernel_643/3231589042.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  pr_6['target_month'] = 0
/var/folders/tf/3bx8pk1n20s1ls73n0mjlrf40000gn/T/ipykernel_643/3231589042.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  pr_6['target_day'] = 0


In [168]:
pr_6.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 93846 entries, 0 to 93845
Data columns (total 5 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   wagnum        93846 non-null  int64  
 1   month         93846 non-null  object 
 2   A             93846 non-null  float64
 3   target_month  93846 non-null  int64  
 4   target_day    93846 non-null  int64  
dtypes: float64(1), int64(3), object(1)
memory usage: 3.6+ MB


In [210]:
pr_6.loc[(pr_6['A'] == 2), 'target_month'] = 1
pr_6.loc[(pr_6['A'] == 2), 'target_day'] = 1
pr_6.loc[(pr_6['A'] == 1), 'target_month'] = 1

In [211]:
pr_7 = pr_6.drop('A', axis=1)

In [171]:
pr_7

,wagnum,month,target_month,target_day
0,33361,2023-03-01,1,1
1,33364,2023-03-01,1,1
2,33366,2023-03-01,1,1
3,33358,2023-03-01,1,1
4,33349,2023-03-01,1,1
...,...,...,...,...
93841,17621,2023-03-01,0,0
93842,25045,2023-03-01,1,1
93843,27156,2023-03-01,1,1
93844,21361,2023-03-01,1,1


In [212]:
p = y_predict_submit_example.merge(pr_7, how='left', on='wagnum')

In [189]:
p1 = p.iloc[:33707,:].copy()
p1['month'] = p1['month_x']
p1['target_month'] = p1['target_month_y']
p1['target_day'] = p1['target_day_y']

In [213]:
p1

,wagnum,month,target_month,target_day
0,33361,2023-03-01,1,1
1,33364,2023-03-01,1,1
2,33366,2023-03-01,1,1
3,33358,2023-03-01,1,1
4,33349,2023-03-01,1,1
...,...,...,...,...
33702,2088,2023-03-01,1,1
33703,2088,2023-03-01,1,1
33704,2088,2023-03-01,1,1
33705,2088,2023-03-01,1,1


In [191]:
p1.drop(['month_y','month_x','target_month_x','target_day_y','target_month_x','target_month_y','target_day_x'], axis=1, inplace=True)

In [192]:
p1

,wagnum,month,target_month,target_day
0,33361,2023-03-01,1,1
1,33364,2023-03-01,1,1
2,33366,2023-03-01,1,1
3,33358,2023-03-01,1,1
4,33349,2023-03-01,1,1
...,...,...,...,...
33702,2088,2023-03-01,1,1
33703,2088,2023-03-01,1,1
33704,2088,2023-03-01,1,1
33705,2088,2023-03-01,1,1


In [193]:
p1.to_csv('pr.csv', index=False)

In [ ]:
pr_7.to_csv('output.csv', index=False, dtype={'wagnum': 'int63','month':'object',''})